In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
file_path = '../input/fashion-data-analytics/FashionData.xlsx'

In [3]:
df  = pd.read_excel(file_path, header = [1], na_values = ['null', 'undefined', ''])

In [4]:
print('Dataset has {} rows & {} columns'.format(*df.shape))
print('Column Names:', df.columns)

Dataset has 49999 rows & 15 columns
Column Names: Index(['timestamp', 'user ID', 'ip_address', 'Product_Name', 'Product_ID',
       'Is_First_Order', 'User_Gender', 'Payment_Type', 'Number_of_Products',
       'Order_Coupon_Code', 'City', 'Country_Province', 'User_Birthday',
       'Country', 'Revenue'],
      dtype='object')


In [5]:
df.dtypes

timestamp             datetime64[ns]
user ID                        int64
ip_address                    object
Product_Name                  object
Product_ID                    object
Is_First_Order               float64
User_Gender                   object
Payment_Type                  object
Number_of_Products            object
Order_Coupon_Code             object
City                          object
Country_Province              object
User_Birthday         datetime64[ns]
Country                       object
Revenue                      float64
dtype: object

In [6]:
print('start date:', df['timestamp'].min(), '& end date:', df['timestamp'].max())
print('Thats 21 days (3 weeks) of customer order/purchase data')

start date: 2017-10-01 00:00:23 & end date: 2017-10-21 23:59:50
Thats 21 days (3 weeks) of customer order/purchase data


In [7]:
df.isnull().sum()

timestamp                 0
user ID                   0
ip_address                0
Product_Name             15
Product_ID               15
Is_First_Order           15
User_Gender            1189
Payment_Type             15
Number_of_Products       15
Order_Coupon_Code     38120
City                    352
Country_Province       4195
User_Birthday         27290
Country                  15
Revenue                  15
dtype: int64

In [8]:
df = df.dropna(subset = ['Product_Name', 'Product_ID', 'Revenue']).reset_index(drop = 1)

In [9]:
split_cols = ['Product_Name', 'Product_ID', 'Number_of_Products']
df[split_cols] = df[split_cols].astype('string')

In [10]:
df['Order_Id'] = df.index + 1
df_split = df.apply(lambda x: x.str.split(',') if x.name in split_cols else x)

In [11]:
len(np.where(df_split['Product_ID'].str.len() != df_split['Number_of_Products'].str.len())[0])

16

In [12]:
len(np.where(df_split['Product_Name'].str.len() != df_split['Product_ID'].str.len())[0])

275

In [13]:
len(np.where(df_split['Product_Name'].str.len() != df_split['Number_of_Products'].str.len())[0])

275

In [14]:
def fill_missing_splits(row):
    num_pqs = len(row['Number_of_Products'])
    num_pid = len(row['Product_ID'])
    
    if num_pqs > num_pid:
        missing_val = ['missingPID' for i in range(num_pid,num_pqs)]
        row['Product_ID'].extend(missing_val)
        
    elif num_pqs < num_pid:
        missing_val = ['1' for i in range(num_pqs,num_pid)]
        row['Number_of_Products'].extend(missing_val)
        
    num_pid = len(row['Product_ID'])
    num_pns = len(row['Product_Name'])

    if num_pns < num_pid:
        missing_val = [row['Product_ID'][i] for i in range(num_pns,num_pid)]
        row['Product_Name'].extend(missing_val)
        
    elif num_pns > num_pid:
        row['Product_Name'] = row['Product_Name'][-num_pid:]
    
    return row

In [15]:
df_clean = df_split.apply(fill_missing_splits, axis = 1)

In [16]:
len(np.where(df_clean['Product_ID'].str.len() != df_clean['Number_of_Products'].str.len())[0])

0

In [17]:
len(np.where(df_clean['Product_Name'].str.len() != df_clean['Product_ID'].str.len())[0])

0

In [18]:
len(np.where(df_clean['Product_Name'].str.len() != df_clean['Number_of_Products'].str.len())[0])

0

In [19]:
df_clean[split_cols] = df_clean[split_cols].applymap(lambda x: ','.join(x))

In [20]:
df_clean.rename(columns = {'user ID': 'user_ID'}, inplace = True)

In [21]:
df_clean.shape, df_clean.columns

((49984, 16),
 Index(['timestamp', 'user_ID', 'ip_address', 'Product_Name', 'Product_ID',
        'Is_First_Order', 'User_Gender', 'Payment_Type', 'Number_of_Products',
        'Order_Coupon_Code', 'City', 'Country_Province', 'User_Birthday',
        'Country', 'Revenue', 'Order_Id'],
       dtype='object'))

In [22]:
df_clean.to_csv('FashionData_Clean.csv', index = False)